# Importe De Librerías

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn
import pickle
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from six import StringIO
import pydotplus
from IPython.display import Image  
from sklearn.tree import export_graphviz

import warnings
warnings.filterwarnings('always')
from sklearn.linear_model import LinearRegression
from sklearn import tree, preprocessing, model_selection, ensemble

# Regresión

### Lectura de archivos

In [5]:
ds_regresion = pd.read_csv("properati_argentina_train.csv")
test = pd.read_csv("properati_argentina_test.csv")

ds_regresion_reducido = pd.read_csv("properati_argentina_reducido_train.csv")
test_reducido = pd.read_csv("properati_argentina_reducido_test.csv")

variables_a_eliminar = ["Unnamed: 0"]

ds_regresion.drop(columns=variables_a_eliminar, inplace=True)
test.drop(columns = variables_a_eliminar, inplace=True)

ds_regresion_reducido.drop(columns=variables_a_eliminar, inplace=True)
test_reducido.drop(columns = variables_a_eliminar, inplace=True)

## KNN

### Preparo los datasets


In [8]:
ds_train_knn = ds_regresion.copy()
ds_test_knn = test.copy()

ds_train_reducido_knn = ds_regresion_reducido.copy()
ds_test_reducido_knn = test_reducido.copy()

#### Primero entrenamos el modelo con el dataset no reducido

#### Z-Score - Ingeniería De Características

Escalamos las variables para que las variables con valores más grandes, como 'property surface total', no tengan más peso que las más pequeñas. Esto podría perjudicar las predicciones del modelo, ya que dificlta la comparación de features. Utilizamos Z-Score porque las variables 'property_surface_covered' y 'property_surface_total' no tienen un máximo definido. Podría aparecer una propiedad con terreno muy grande, lo cual  es un problema si normalizamos usando Min-Max. Como Z-Score mide el desvío estándar, no se ve afectado. Esto además lo vimos reflejado en las métricas, las cuales dieron mejor cuando normalizamos usando Z-Score. Para las variables 'property_rooms', 'property_bedrooms' y las resultantes de hacer One Hot Encoding a 'property_type' no fue necesario normalizarlas con Z-Score. No son lo suficientemente grandes como para perjudicar la comparación de features. Esto lo vimos reflejado, en las metricas, las cuales no variaron al aplicarle Z-Score a estas variables

In [9]:
# Escalamos las variables
standard_scaler = StandardScaler()

ds_train_knn["property_surface_covered"] = standard_scaler.fit_transform(ds_train_knn["property_surface_covered"].to_frame())
ds_train_knn["property_surface_total"] = standard_scaler.fit_transform(ds_train_knn["property_surface_total"].to_frame())
ds_train_knn["longitud"] = standard_scaler.fit_transform(ds_train_knn["longitud"].to_frame())
ds_train_knn["latitud"] = standard_scaler.fit_transform(ds_train_knn["latitud"].to_frame())

ds_test_knn["property_surface_covered"] = standard_scaler.transform(ds_test_knn["property_surface_covered"].to_frame())
ds_test_knn["property_surface_total"] = standard_scaler.transform(ds_test_knn["property_surface_total"].to_frame())
ds_test_knn["longitud"] = standard_scaler.transform(ds_test_knn["longitud"].to_frame())
ds_test_knn["latitud"] = standard_scaler.transform(ds_test_knn["latitud"].to_frame())

#### One Hot Encoding - Ingeniería De Características

Para poder correr el algoritmo, relizamos One Hot Encoding para las variables categoricas del dataset

In [97]:
#Realizamos One Hot Encoding

ds_train_knn = pd.get_dummies(ds_train_knn, columns=["property_type"], drop_first=True)
ds_test_knn = pd.get_dummies(ds_test_knn, columns=["property_type"], drop_first=True)

#### Eliminamos variables

Eliminamos la variable place_l3, ya que realizar One Hot Encoding generaría una nueva variable por cada barrio, lo cual agrandaría demasiado la dimensionalidad del dataset. Esto incrementaba el tiempo de entrenamiento del modelo, y no había una ganancia significativa en la performance

In [98]:
#Eliminamos la variable place_l3

variables_eliminadas=["place_l3"]
ds_train_knn.drop(variables_eliminadas, axis='columns', inplace=True)
ds_test_knn.drop(variables_eliminadas, axis='columns', inplace=True)

In [99]:
#Estado actual del dataset 
ds_train_knn


,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_type_Departamento,property_type_PH
0,-0.212018,-0.837706,2.0,1.0,-0.356237,-0.417925,80000.0,1,0
1,0.628267,0.364281,2.0,1.0,-0.609345,-0.762050,79900.0,1,0
2,1.735031,-0.367399,1.0,1.0,-0.749961,-1.130756,69000.0,1,0
3,-1.391398,-0.700909,5.0,3.0,-0.046882,0.221165,150000.0,1,0
4,1.820228,-0.653623,2.0,1.0,-0.454668,-0.786631,85000.0,1,0
...,...,...,...,...,...,...,...,...,...
59736,-1.342367,1.818753,1.0,1.0,-0.595284,-0.835791,70000.0,1,0
59737,1.502567,-0.676071,3.0,2.0,-0.046882,0.221165,158000.0,1,0
59738,-1.253022,0.660575,3.0,2.0,0.304658,0.344067,175000.0,1,0
59739,0.348525,-0.362046,2.0,1.0,-0.243744,-0.172121,122000.0,1,0


#### Optimización de parametros con Random Search

In [100]:
x_train = ds_train_knn.drop(columns=["property_price"])
y_train = ds_train_knn["property_price"]

Decidimos usar Random Search para la búsqueda de hiperparametros. Si bien Grid Search permite buscar más combinaciones, su tiempo de ejecución puede ser demasiado alto, y no nos parece que se justifique su utilización. Random Search es capaz de darnos hiperparametros que generan buenas métricas con menos timepo de ejecución. Usamos 5 folds. Conluimos que usar más folds no mejora la performance, pero aumenta el tiempo de ejecución

In [101]:
#Grilla de Parámetros
params_grid={ 'n_neighbors':range(1,30), 
              'weights':['distance','uniform'],
              'algorithm':['ball_tree', 'kd_tree', 'brute'],
              'metric':['euclidean','manhattan','chebyshev', 'minkowski'],
              'leaf_size': list(range(0, 50)),
             }

#Metrica que quiero optimizar MSE
scorer_fn = make_scorer(sklearn.metrics.mean_squared_error)

#Clasificador KNN
knn=KNeighborsRegressor()

#Random Search con 5 Folds y 10 iteraciones
rand = RandomizedSearchCV(knn, params_grid, cv=5, scoring=scorer_fn, n_iter=10, random_state=5)

rand.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(),
                   param_distributions={'algorithm': ['ball_tree', 'kd_tree',
                                                      'brute'],
                                        'leaf_size': [0, 1, 2, 3, 4, 5, 6, 7, 8,
                                                      9, 10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, ...],
                                        'metric': ['euclidean', 'manhattan',
                                                   'chebyshev', 'minkowski'],
                                        'n_neighbors': range(1, 30),
                                        'weights': ['distance', 'uniform']},
                   random_state=5, scoring=make_scorer(mean_squared_error))

In [102]:
#Mejores hiperparametros
print(rand.best_params_)

#Mejor métrica
mse = rand.best_score_

print("MSE en datos de entrnamiento: " + str((mse)))
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))

{'weights': 'uniform', 'n_neighbors': 24, 'metric': 'euclidean', 'leaf_size': 38, 'algorithm': 'kd_tree'}
MSE en datos de entrnamiento: 3341920112.489032
RMSE en datos de entrnamiento: 57809.34277855987


In [103]:
#Mejor estimador
best_knn=rand.best_estimator_

x_test_knn = ds_test_knn.drop(columns=["property_price"])
y_test_knn = ds_test_knn["property_price"]

y_pred_knn = best_knn.predict(x_test_knn)

In [104]:
#Metricas para evaluar modelos
from sklearn import metrics

#Mean Square Error
rmse = metrics.mean_squared_error(
        y_true  = y_test_knn,
        y_pred  = y_pred_knn,
        squared = True
       )

#Root Mean Square Error
rmse = metrics.mean_squared_error(
        y_true  = y_test_knn,
        y_pred  = y_pred_knn,
        squared = False
       )

print(f"El error (mse) de test es: {mse}")
print(f"El error (rmse) de test es: {rmse}")

El error (mse) de test es: 3341920112.489032
El error (rmse) de test es: 58628.5692322967


Buscamos los mejores hiperparametros que optimizen el MSE, ya que es la medida que nos interesa a al hora de hacer regresion. Queremos calcular el error medio de las predicciones, cuanto menor sea el error, mejor será la performance del modelo.

La metrica que utlizamos para evaluar el modelo es el RMSE. La ventaja de RMSE sobre MSE es que los valores de la métrica se corresponden con las unidades de la variable target que se predice. Así es más facil de dimensionar y entender cuanto es el error. 

La performance del modelo en el conjunto de entrenamiento fue muy similar a la de testeo. Esto significa que no hay overfitting, de haberlo la performance en el conjunto de entrenamiento sería mucho mejor que en el de testeo. Tampoco creemos que haya underfitting. Un RMSE de aproximadamente 60.000 tanto en conjunto de test como en el de entrenamiento, si bien no creemos que sea extremadamente bajo, no lo consideramos suficiente como para afirmar que el modelo sufre de underfitting. Dado esto consideramos que es un modelo robusto.

Guardamos el modelo

In [105]:
filename = "knn_normal.sav"
pickle.dump(best_knn, open(filename, 'wb'))

<ipython-input-105-27570a3e5224>:2: ResourceWarning: unclosed file <_io.BufferedWriter name='knn_normal.sav'>
  pickle.dump(best_knn, open(filename, 'wb'))


#### Ahora entrenamos el modelo con el dataset reducido

In [106]:
#Estado del dataset reducido
ds_train_reducido_knn.head()

,cp_1,cp_2,cp_3,cp_4,cp_5,cp_6,target
0,-1.209465,-0.279373,0.403256,0.198202,-0.675469,0.346012,80000.0
1,-1.541488,0.093449,0.236567,-0.004881,0.579156,-0.184825,79900.0
2,-2.208671,-0.215462,1.539594,-0.077990,0.826223,-0.156335,69000.0
3,1.659969,0.709044,-0.613635,-0.004372,-2.038534,-0.763029,150000.0
4,-1.496299,-0.003512,1.798436,-0.161402,0.592708,-0.151901,85000.0


#### Optimizacion de parametros con Random search

In [107]:
x_train = ds_train_reducido_knn.drop(columns=["target"])
y_train = ds_train_reducido_knn["target"]

Nuevamente, y por las razones ya mencionadas más arriba, utilizamos Random Search y no Grid Search. Usamos 5 K-Folds, por las razones ya explicadas previamente

In [108]:
#Grilla de Parámetros
params_grid={ 'n_neighbors':range(1,30), 
              'weights':['distance','uniform'],
              'algorithm':['ball_tree', 'kd_tree', 'brute'],
              'metric':['euclidean','manhattan','chebyshev', 'minkowski'],
              'leaf_size': list(range(0, 50))
             }

#Metrica que quiero optimizar MSE
scorer_fn = make_scorer(sklearn.metrics.mean_squared_error)

#Clasificador KNN
knn=KNeighborsRegressor()

#Random Search con 10 Folds y 10 iteraciones
rand = RandomizedSearchCV(knn, params_grid, cv=5, scoring=scorer_fn, n_iter=10, random_state=5)

rand.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(),
                   param_distributions={'algorithm': ['ball_tree', 'kd_tree',
                                                      'brute'],
                                        'leaf_size': [0, 1, 2, 3, 4, 5, 6, 7, 8,
                                                      9, 10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, ...],
                                        'metric': ['euclidean', 'manhattan',
                                                   'chebyshev', 'minkowski'],
                                        'n_neighbors': range(1, 30),
                                        'weights': ['distance', 'uniform']},
                   random_state=5, scoring=make_scorer(mean_squared_error))

In [109]:
#Mejores hiperparametros
print(rand.best_params_)

#Mejor métrica
mse = rand.best_score_
print("MSE en datos de entrnamiento: " + str((mse)))
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))


{'weights': 'uniform', 'n_neighbors': 24, 'metric': 'euclidean', 'leaf_size': 38, 'algorithm': 'kd_tree'}
MSE en datos de entrnamiento: 3264130628.9708567
RMSE en datos de entrnamiento: 57132.57064906896


In [110]:
#Mejor estimador
best_knn=rand.best_estimator_

x_test_reducido_knn = ds_test_reducido_knn.drop(columns=["target"])
y_test_reducido_knn = ds_test_reducido_knn["target"]

y_pred_knn = best_knn.predict(x_test_reducido_knn)

In [111]:
#Metricas para evaluar modelos
from sklearn import metrics

#Mean Squared Error
mse = metrics.mean_squared_error(
        y_true  = y_test_reducido_knn,
        y_pred  = y_pred_knn,
        squared = True
       )


#Root Mean Square Error
rmse = metrics.mean_squared_error(
        y_true  = y_test_reducido_knn,
        y_pred  = y_pred_knn,
        squared = False
       )


print(f"El error (mse) de test es: {mse}")
print(f"El error (rmse) de test es: {rmse}")

El error (mse) de test es: 3030461879.570157
El error (rmse) de test es: 55049.63105752987


Nuevamente, y por las mismas razones, buscamos los hiperparametros que optimizen MSE. A la hora de analizar la performance, volvemos a utilizar RMSE por la razones mencionadas previamente

Las metricas en los conjuntos de test y entrenamiento fueron similares. Consideramos que es un buen modelo, donde no se overfitea. Si bien hay margen de mejora para las métricas, las considermos como aceptables. 

Si comparamos los dos modelos (KNN con dataset reducido y KNN con dataset sin reducir) vemos que son bastante similares. Ambos modelos son buenos, con el modelo que usa el dataset reducido dando métricas apenas más bajas. La mayor diferencia a notamos en el tiempo de entrenamiento. Con el dataset reducido el tiempo de ejecución fue menor

Persistimos el modelo

In [112]:
filename = "knn_reducido.sav"
pickle.dump(best_knn, open(filename, 'wb'))

<ipython-input-112-cb20c830b1b2>:2: ResourceWarning: unclosed file <_io.BufferedWriter name='knn_reducido.sav'>
  pickle.dump(best_knn, open(filename, 'wb'))


## XGBoost

In [113]:
ds_prop_XGBoost_train = ds_regresion.copy()
ds_prop_XGBoost_test = test.copy()

ds_prop_XGBoost_reducido_train = ds_regresion_reducido.copy()
ds_prop_XGBoost_reducido_test = test_reducido.copy()


#### Primero entrenamos el modelo con el dataset sin reducir

#### Ingeniería De Caraterísticas

Z-Score - Ingeniería De Características

Nuevamente, y por las mismas razones mencionadas en el algoritmo KNN, usamos Z-Score. Volvimos a notar que la performance mejora al usar Z-Score en vez de Min-Max. No normalizamos 'property_rooms', ni 'property_bedrooms', las variables de 'property_type', por las razones ya mencionadas (no mejora la performance)

In [114]:
#Escalamos las variables para que no tengan mayor peso
standard_scaler = StandardScaler()

ds_prop_XGBoost_train["property_surface_covered"] = standard_scaler.fit_transform(ds_prop_XGBoost_train["property_surface_covered"].to_frame())
ds_prop_XGBoost_train["property_surface_total"] = standard_scaler.fit_transform(ds_prop_XGBoost_train["property_surface_total"].to_frame())
ds_prop_XGBoost_train["longitud"] = standard_scaler.fit_transform(ds_prop_XGBoost_train["longitud"].to_frame())
ds_prop_XGBoost_train["latitud"] = standard_scaler.fit_transform(ds_prop_XGBoost_train["latitud"].to_frame())

ds_prop_XGBoost_test["property_surface_covered"] = standard_scaler.transform(ds_prop_XGBoost_test["property_surface_covered"].to_frame())
ds_prop_XGBoost_test["property_surface_total"] = standard_scaler.transform(ds_prop_XGBoost_test["property_surface_total"].to_frame())
ds_prop_XGBoost_test["longitud"] = standard_scaler.transform(ds_prop_XGBoost_test["longitud"].to_frame())
ds_prop_XGBoost_test["latitud"] = standard_scaler.transform(ds_prop_XGBoost_test["latitud"].to_frame())

#### One Hot Encoding - Ingeniería De Características

Por las razones ya mencionadas en KNN, usamos One Hot Encoding para las variables categoricas

In [115]:
#One Hot Encoding para variables categoricas

variables_reemplazadas = ["property_type"]
ds_prop_XGBoost_train = pd.get_dummies(ds_prop_XGBoost_train, columns=variables_reemplazadas, drop_first=True)
ds_prop_XGBoost_test = pd.get_dummies(ds_prop_XGBoost_test, columns=variables_reemplazadas, drop_first=True)

#### Eliminamos Variables

Volvemos a eliminar la variable 'place_l3' por las razones explicadas previamente

In [116]:
variables_eliminadas=["place_l3"]
ds_prop_XGBoost_train.drop(variables_eliminadas, axis='columns', inplace=True)
ds_prop_XGBoost_test.drop(variables_eliminadas, axis='columns', inplace=True)

In [117]:
#Estado del dataset
ds_prop_XGBoost_train

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_type_Departamento,property_type_PH
0,-0.212018,-0.837706,2.0,1.0,-0.356237,-0.417925,80000.0,1,0
1,0.628267,0.364281,2.0,1.0,-0.609345,-0.762050,79900.0,1,0
2,1.735031,-0.367399,1.0,1.0,-0.749961,-1.130756,69000.0,1,0
3,-1.391398,-0.700909,5.0,3.0,-0.046882,0.221165,150000.0,1,0
4,1.820228,-0.653623,2.0,1.0,-0.454668,-0.786631,85000.0,1,0
...,...,...,...,...,...,...,...,...,...
59736,-1.342367,1.818753,1.0,1.0,-0.595284,-0.835791,70000.0,1,0
59737,1.502567,-0.676071,3.0,2.0,-0.046882,0.221165,158000.0,1,0
59738,-1.253022,0.660575,3.0,2.0,0.304658,0.344067,175000.0,1,0
59739,0.348525,-0.362046,2.0,1.0,-0.243744,-0.172121,122000.0,1,0


#### Optimización de parametros con Random Search

In [118]:
features = list(ds_prop_XGBoost_train.columns.values)
features.pop(features.index("property_price"))

target = ["property_price"]

x_train = ds_prop_XGBoost_train[features]
x_test = ds_prop_XGBoost_test[features]

y_train = ds_prop_XGBoost_train[target]
y_test = ds_prop_XGBoost_test[target]

Nuevamente, por las razones ya explicadas, buscamos los mejores hiperparametros usando Random CV. Usamos 8 folds por las razones ya explicadas

In [119]:
#KFOLD CV Random Search para buscar el mejor arbol (los mejores atributos, hiperparametros,etc)

#Cantidad de combinaciones que quiero porbar
n=10

#Conjunto de parámetros que quiero usar
params_grid = {'criterion':['gini','entropy'],                  #Luego de probar con varias combinaciones de parametros se
               'ccp_alpha':np.linspace(0,0.5,15),               #llegó a la conclusión de que Random Search encuentra los
               'max_depth':list(range(3,4)),                    #hiperparametros más adecuados con esta grilla 
               'random_state':list(range(0,6)),                 #de parametros
               'gamma':list(range(31,32)),
               'min_child_weight':list(range(17,18)),
               'colsample_bytree':np.linspace(0.75,0.8,15)}
                
#Cantidad de splits para el Cross Validation
folds=8

#Regresor
xgb_model_rd_search = xgb.XGBRegressor()

#Metrica que quiero optimizar MSE
scorer_fn = make_scorer(sklearn.metrics.mean_squared_error)

#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=xgb_model_rd_search,
                              param_distributions = params_grid,
                              scoring=scorer_fn,
                              n_iter=n, cv=folds, random_state=5) 


#Busco los hiperparamtros que optimizan MSE
randomcv.fit(x_train,y_train)

[23:00:06] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:00:07] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:00:09] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:00:10] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:00:11] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:00:12] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:00:13] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:00:14] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:00:16] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:00:17] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.



RandomizedSearchCV(cv=8,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
       0.35714286, 0.39285714, 0.42857143, 0.46428571, 0.5       ]),
                                        'colsample_bytree': array([0.75      , 0.75357143, 0.75714286, 0.76071429, 0.76428571,
       0.76785714, 0.77142857, 0.775     , 0.77857143, 0.78214286,
       0.78571429, 0.78928571, 0.79285714, 0.79642857, 0.8       ]),
                                        'criterion': ['gini', 'entropy'],
                                        'gamma': [31], 'max_depth': [3],
                                        'min_child_weight': [17],
                                        'random_state': [0, 1, 2, 3, 4, 5]},
                   random_state=5, scoring=make_scorer(mean_squared_error))

In [120]:
#Mejores hiperparametros
print(randomcv.best_params_)

#Mejor métrica
mse = randomcv.best_score_

print("MSE en datos de entrnamiento: " + str((mse)))
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))

{'random_state': 4, 'min_child_weight': 17, 'max_depth': 3, 'gamma': 31, 'criterion': 'entropy', 'colsample_bytree': 0.775, 'ccp_alpha': 0.42857142857142855}
MSE en datos de entrnamiento: 3136115717.570307
RMSE en datos de entrnamiento: 56001.03318306107


#### Entrenamos el modelo

In [121]:
xgb_model = xgb.XGBRegressor().set_params(**randomcv.best_params_)
xgb_model.fit(x_train, y_train)

[23:01:44] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.



XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             ccp_alpha=0.42857142857142855, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.775, criterion='entropy',
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=31, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=3, max_leaves=0, min_child_weight=17, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, ...)

In [122]:
from sklearn.metrics import mean_squared_error

y_pred = xgb_model.predict(x_test)

mse = mean_squared_error(y_test, y_pred, squared=True)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("MSE en datos de test: " + str(mse))
print("RMSE en datos de test: " + str(rmse))

MSE en datos de test: 3527173265.411116
RMSE en datos de test: 59390.00981150884


In [123]:
#Atributos considerados y su importancia
sorted(list(zip(x_train.columns.to_list(), xgb_model.feature_importances_)), key=lambda x: -x[1])

[('property_surface_covered', 0.3532839),
 ('property_type_Departamento', 0.24006331),
 ('property_surface_total', 0.12723376),
 ('property_rooms', 0.107419446),
 ('property_type_PH', 0.06379977),
 ('latitud', 0.057206),
 ('longitud', 0.041771196),
 ('property_bedrooms', 0.0092226425)]

Volvemos a buscar los hiperparametros que optimizen MSE. Al ser hacer regresión, la métrica más conveniente es MSE, queremos el error cuadrático medio 
Nuevamente medimos que tan precisas fueron las predicciones usando RMSE. La ventaja de usar RMSE en vez de MSE es que el valor de RMSE está en las mismas unidades que la variable target

La performance del conjunto de test y de entrenamiento fue similar, lo cual es bueno. El hecho de que las performances sean sean similares nos indica que no hay overfitting. Las metricas en ambos conjuntos no son altas, por lo tanto no consideramos que haya underfitting

Guardamos el modelo

In [124]:
filename = "xgb_model_normal.sav"
pickle.dump(xgb_model, open(filename, 'wb'))

<ipython-input-124-9c22e7b7604a>:2: ResourceWarning: unclosed file <_io.BufferedWriter name='xgb_model_normal.sav'>
  pickle.dump(xgb_model, open(filename, 'wb'))


#### Ahora probamos el modelo con el dataset reducido

In [125]:
x_train = ds_prop_XGBoost_reducido_train.drop(columns=["target"])
y_train = ds_prop_XGBoost_reducido_train["target"]

x_test = ds_prop_XGBoost_reducido_test.drop(columns=["target"])
y_test = ds_prop_XGBoost_reducido_test["target"]

In [126]:
#Estado del dataset reducido
ds_prop_XGBoost_reducido_train

,cp_1,cp_2,cp_3,cp_4,cp_5,cp_6,target
0,-1.209465,-0.279373,0.403256,0.198202,-0.675469,0.346012,80000.0
1,-1.541488,0.093449,0.236567,-0.004881,0.579156,-0.184825,79900.0
2,-2.208671,-0.215462,1.539594,-0.077990,0.826223,-0.156335,69000.0
3,1.659969,0.709044,-0.613635,-0.004372,-2.038534,-0.763029,150000.0
4,-1.496299,-0.003512,1.798436,-0.161402,0.592708,-0.151901,85000.0
...,...,...,...,...,...,...,...
59736,-1.989015,0.036037,-2.168018,0.337857,0.577288,0.292543,70000.0
59737,0.132021,0.622959,1.590603,-0.286189,0.156480,-0.372168,158000.0
59738,0.368774,0.661110,-1.318088,0.119680,-0.471374,0.283828,175000.0
59739,-1.090553,0.023561,0.529813,0.067196,0.005031,0.285692,122000.0


#### Optimización de parametros con Random Search

Volvemos a usar Random Search para buscar los hiperparametros por las razones explicadas previamente. Usamos 8 folds

In [127]:
#KFOLD CV Random Search para buscar el mejor arbol (los mejores atributos, hiperparametros,etc)

#Cantidad de combinaciones que quiero porbar
n=10

#Conjunto de parámetros que quiero usar
params_grid = {'criterion':['gini','entropy'],                  #Luego de probar con varias combinaciones de parametros se
               'ccp_alpha':np.linspace(0,0.5,15),               #llegó a la conclusión de que Random Search encuentra los
               'max_depth':list(range(3,4)),                    #hiperparametros más adecuados con esta grilla 
               'random_state':list(range(0,6)),                 #de parametros
               'gamma':list(range(31,32)),
               'min_child_weight':list(range(17,18)),
               'colsample_bytree':np.linspace(0.75,0.8,15)}
                
#Cantidad de splits para el Cross Validation
folds=8

#Regresor
xgb_model_rd_search = xgb.XGBRegressor()

#Metrica que quiero optimizar MSE
scorer_fn = make_scorer(sklearn.metrics.mean_squared_error)

#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=xgb_model_rd_search,
                              param_distributions = params_grid,
                              scoring=scorer_fn,
                              n_iter=n, cv=folds, random_state=5) 


#Busco los hiperparamtros que optimizan MSE
randomcv.fit(x_train,y_train)

[23:01:45] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:01:47] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:01:50] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:01:51] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:01:53] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:01:54] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:01:56] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:01:57] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:01:59] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[23:02:00] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.



RandomizedSearchCV(cv=8,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
       0.35714286, 0.39285714, 0.42857143, 0.46428571, 0.5       ]),
                                        'colsample_bytree': array([0.75      , 0.75357143, 0.75714286, 0.76071429, 0.76428571,
       0.76785714, 0.77142857, 0.775     , 0.77857143, 0.78214286,
       0.78571429, 0.78928571, 0.79285714, 0.79642857, 0.8       ]),
                                        'criterion': ['gini', 'entropy'],
                                        'gamma': [31], 'max_depth': [3],
                                        'min_child_weight': [17],
                                        'random_state': [0, 1, 2, 3, 4, 5]},
                   random_state=5, scoring=make_scorer(mean_squared_error))

In [128]:
#Mejores hiperparametros
print(randomcv.best_params_)

#Mejor métrica
mse = randomcv.best_score_
print("MSE en datos de entrnamiento: " + str((mse)))
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))

{'random_state': 5, 'min_child_weight': 17, 'max_depth': 3, 'gamma': 31, 'criterion': 'entropy', 'colsample_bytree': 0.7821428571428571, 'ccp_alpha': 0.46428571428571425}
MSE en datos de entrnamiento: 3598429986.8550777
RMSE en datos de entrnamiento: 59986.91513034386


In [129]:
xgb_model = xgb.XGBRegressor().set_params(**randomcv.best_params_)
xgb_model.fit(x_train, y_train)

[23:03:56] WARNING: ../src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.



XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             ccp_alpha=0.46428571428571425, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7821428571428571,
             criterion='entropy', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=31, gpu_id=-1, grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=3, max_leaves=0, min_child_weight=17, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, ...)

Medimos que tan precisas fueron las predicciones usando RMSE. La ventaja de usar RMSE en vez de MSE es que el valor de RMSE está en las mismas unidades que la variable target. De esta forma es más fácil dimensionar que tan preciso es el modelo creado

In [130]:
from sklearn.metrics import mean_squared_error

y_pred = xgb_model.predict(x_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred, squared=True)

print("MSE en datos de test: " + str(mse))
print("RMSE en datos de test: " + str(rmse))

MSE en datos de test: 3444415497.7188416
RMSE en datos de test: 58689.14292881471


In [131]:
#Atributos considerados y su importancia
sorted(list(zip(x_train.columns.to_list(), xgb_model.feature_importances_)), key=lambda x: -x[1])

[('cp_2', 0.36029565),
 ('cp_1', 0.28183162),
 ('cp_4', 0.12917024),
 ('cp_6', 0.08917866),
 ('cp_5', 0.08702603),
 ('cp_3', 0.05249787)]

Vemos que, similar al modelo que usa el dataset sin reducir, la performance de entrenamiento y la del conjunto de evaluación es bastante similar. Esto significa que no hay overfitting, es decir, el modelo no se aprende de 'memoria' los datos de entrenamiento y no pierde precision al realizar las predicciones en el conjunto de evaluación. Consideramos que es un modelo robusto.

No hay muchas diferencias entre el modelo que usa el dataset reducido y el que el dataset sin reducir. Las métricas son muy similares, y los tiempos de ejecución también

Por ultimo, persistimos el modelo

In [132]:
filename = "xgb_model_reducido.sav"
pickle.dump(xgb_model, open(filename, 'wb'))

<ipython-input-132-cd649da7a5de>:2: ResourceWarning: unclosed file <_io.BufferedWriter name='xgb_model_reducido.sav'>
  pickle.dump(xgb_model, open(filename, 'wb'))


## GradientBoost

In [133]:
ds_prop_GBoost_train = ds_regresion.copy()
ds_prop_GBoost_test = test.copy()

ds_prop_GBoost_reducido_train = ds_regresion_reducido.copy()
ds_prop_GBoost_reducido_test = test_reducido.copy()

#### Primero entrenamos el modelo con el dataset sin reducir

#### Ingenieria De Caracteristicas

Z-Score - Ingeniera De Caracteristicas

Por las razones ya mencionadas, volvemos a normalizar usando Z-Score. Al igual que con KNN y XGBoost, notamos como las métricas mejoraron cuando usamos Z-Score para normalizar, en comparación con Min-Max. También, por las mismas razones, no normalizamos 'property_rooms', ni 'property_bedrooms', ni las variables que son producto de realizarle One Hot Encoding a 'propety_type'.

In [134]:
#Escalamos las variables para que no tengan mayor peso

standard_scaler = StandardScaler()

ds_prop_GBoost_train["property_surface_covered"] = standard_scaler.fit_transform(ds_prop_GBoost_train["property_surface_covered"].to_frame())
ds_prop_GBoost_train["property_surface_total"] = standard_scaler.fit_transform(ds_prop_GBoost_train["property_surface_total"].to_frame())
ds_prop_GBoost_train["longitud"] = standard_scaler.fit_transform(ds_prop_GBoost_train["longitud"].to_frame())
ds_prop_GBoost_train["latitud"] = standard_scaler.fit_transform(ds_prop_GBoost_train["latitud"].to_frame())

ds_prop_GBoost_test["property_surface_covered"] = standard_scaler.transform(ds_prop_GBoost_test["property_surface_covered"].to_frame())
ds_prop_GBoost_test["property_surface_total"] = standard_scaler.transform(ds_prop_GBoost_test["property_surface_total"].to_frame())
ds_prop_GBoost_test["longitud"] = standard_scaler.transform(ds_prop_GBoost_test["longitud"].to_frame())
ds_prop_GBoost_test["latitud"] = standard_scaler.transform(ds_prop_GBoost_test["latitud"].to_frame())

#### One Hot Encoding - Ingeniería De Caracteristicas

Volvemos a usar One Hot Encoding para las variables categoricas por las razones explicadas previamente

In [135]:
#One Hot Encoding para variables categoricas

variables_reemplazadas = ["property_type"]
ds_prop_GBoost_train = pd.get_dummies(ds_prop_GBoost_train, columns=variables_reemplazadas, drop_first=True)
ds_prop_GBoost_test = pd.get_dummies(ds_prop_GBoost_test, columns=variables_reemplazadas, drop_first=True)

#### Eliminamos Variables

Nuevamente, por las mismas razones, volvemos a eliminar la variable 'place_l3'

In [136]:
#Eliminamos place_l3 

variables_eliminadas=["place_l3"]
ds_prop_GBoost_train.drop(variables_eliminadas, axis='columns', inplace=True)
ds_prop_GBoost_test.drop(variables_eliminadas, axis='columns', inplace=True)

In [137]:
#Estado del dataset

ds_prop_GBoost_train

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_type_Departamento,property_type_PH
0,-0.212018,-0.837706,2.0,1.0,-0.356237,-0.417925,80000.0,1,0
1,0.628267,0.364281,2.0,1.0,-0.609345,-0.762050,79900.0,1,0
2,1.735031,-0.367399,1.0,1.0,-0.749961,-1.130756,69000.0,1,0
3,-1.391398,-0.700909,5.0,3.0,-0.046882,0.221165,150000.0,1,0
4,1.820228,-0.653623,2.0,1.0,-0.454668,-0.786631,85000.0,1,0
...,...,...,...,...,...,...,...,...,...
59736,-1.342367,1.818753,1.0,1.0,-0.595284,-0.835791,70000.0,1,0
59737,1.502567,-0.676071,3.0,2.0,-0.046882,0.221165,158000.0,1,0
59738,-1.253022,0.660575,3.0,2.0,0.304658,0.344067,175000.0,1,0
59739,0.348525,-0.362046,2.0,1.0,-0.243744,-0.172121,122000.0,1,0


#### Optimización de parametros con Random Search

Volvemos a usar Random Search, esta vez con 8 folds. Las razones son las mismas que fueron explicadas previamente

In [138]:
#Hacemos division Train-Test

x_train = ds_prop_GBoost_train.drop(columns=["property_price"])
y_train = ds_prop_GBoost_train["property_price"]

x_test = ds_prop_GBoost_test.drop(columns=["property_price"])
y_test = ds_prop_GBoost_test["property_price"]

In [139]:
#KFOLD CV Random Search para buscar el mejor arbol (los mejores atributos, hiperparametros,etc)

#Cantidad de combinaciones que quiero porbar
n=10

#Conjunto de parámetros que quiero usar
params_grid = {'n_estimators': [50, 100], #nº de etapas de boosting
                        'learning_rate': [0.01, 0.1], #reduce la contribucion de cada arbol por este valor 
                        'max_features': [4, 5], #nº de variables a tener en cuenta para las divisiones
                        'min_samples_split': [5, 10]} #nº mínimo de observaciones necesarias para dividir un nodo interno (n.minobsinnode en R)

                
#Cantidad de splits para el Cross Validation
folds=8

#Regresor
gb_model_rd_search = ensemble.GradientBoostingRegressor()

#Metrica que quiero optimizar MSE
scorer_fn = make_scorer(sklearn.metrics.mean_squared_error)

#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=gb_model_rd_search,
                              param_distributions = params_grid,
                              scoring=scorer_fn,
                              n_iter=n, cv=folds, random_state=5) 


#Busco los hiperparamtros que optimizan MSE
randomcv.fit(x_train,y_train)

RandomizedSearchCV(cv=8, estimator=GradientBoostingRegressor(),
                   param_distributions={'learning_rate': [0.01, 0.1],
                                        'max_features': [4, 5],
                                        'min_samples_split': [5, 10],
                                        'n_estimators': [50, 100]},
                   random_state=5, scoring=make_scorer(mean_squared_error))

In [140]:
#Mejores hiperparametros
print(randomcv.best_params_)

#Mejor métrica
mse = randomcv.best_score_
print("MSE en datos de entrnamiento: " + str((mse)))
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))

{'n_estimators': 50, 'min_samples_split': 10, 'max_features': 4, 'learning_rate': 0.01}
MSE en datos de entrnamiento: 10886244191.881912
RMSE en datos de entrnamiento: 104337.16591839129


#### Entrenamos el modelo

In [141]:
gb_model = ensemble.GradientBoostingRegressor().set_params(**randomcv.best_params_)
gb_model.fit(x_train, y_train)

GradientBoostingRegressor(learning_rate=0.01, max_features=4,
                          min_samples_split=10, n_estimators=50)

In [142]:
from sklearn.metrics import mean_squared_error

y_pred = gb_model.predict(x_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred, squared=True)

print("MSE en datos de test: " + str(mse))
print("RMSE en datos de test: " + str(rmse))

MSE en datos de test: 10767214401.12218
RMSE en datos de test: 103765.18877312458


In [143]:
#Atributos considerados y su importancia
sorted(list(zip(x_train.columns.to_list(), gb_model.feature_importances_)), key=lambda x: -x[1])

[('property_surface_covered', 0.5020011700616139),
 ('property_surface_total', 0.2652998428597314),
 ('property_rooms', 0.06838196207057666),
 ('property_bedrooms', 0.045130877466310115),
 ('property_type_Departamento', 0.0417256483333209),
 ('latitud', 0.03976336195046516),
 ('longitud', 0.027611700818689758),
 ('property_type_PH', 0.010085436439292036)]

Volvemos a buscar los hiperparámetros que optimicen la MSE, y nuevamente usamos el RMSE para analizar la performance de modelo.

La performance del modelo fue muy similar entre los datos del conjunto de entrenamiento y el conjunto de test. Por lo tanto, consideramos que no hay overfitting. Las métricas son un poco más elevadas con respecto a los modelos que usan KNN y XGBoost, por lo tanto es evidente que el modelo no ajusta igual de bien que con los otros dos algoritmos

Guardamos el modelo

In [144]:
filename = "gradient_boost_completo.sav"
pickle.dump(gb_model, open(filename, 'wb'))

<ipython-input-144-a44c18629561>:2: ResourceWarning: unclosed file <_io.BufferedWriter name='gradient_boost_completo.sav'>
  pickle.dump(gb_model, open(filename, 'wb'))


#### Ahora usamos el dataset reducido

In [145]:
x_train = ds_prop_GBoost_reducido_train.drop(columns=["target"])
y_train = ds_prop_GBoost_reducido_train["target"]

x_test = ds_prop_GBoost_reducido_test.drop(columns=["target"])
y_test = ds_prop_GBoost_reducido_test["target"]


In [146]:
ds_prop_GBoost_reducido_train

,cp_1,cp_2,cp_3,cp_4,cp_5,cp_6,target
0,-1.209465,-0.279373,0.403256,0.198202,-0.675469,0.346012,80000.0
1,-1.541488,0.093449,0.236567,-0.004881,0.579156,-0.184825,79900.0
2,-2.208671,-0.215462,1.539594,-0.077990,0.826223,-0.156335,69000.0
3,1.659969,0.709044,-0.613635,-0.004372,-2.038534,-0.763029,150000.0
4,-1.496299,-0.003512,1.798436,-0.161402,0.592708,-0.151901,85000.0
...,...,...,...,...,...,...,...
59736,-1.989015,0.036037,-2.168018,0.337857,0.577288,0.292543,70000.0
59737,0.132021,0.622959,1.590603,-0.286189,0.156480,-0.372168,158000.0
59738,0.368774,0.661110,-1.318088,0.119680,-0.471374,0.283828,175000.0
59739,-1.090553,0.023561,0.529813,0.067196,0.005031,0.285692,122000.0


#### Optimización de parametros con Random Search

Usamos Random Search con 8 folds por las razones explicadas previamente

In [147]:
#KFOLD CV Random Search para buscar el mejor arbol (los mejores atributos, hiperparametros,etc)

#Cantidad de combinaciones que quiero porbar
n=10

#Conjunto de parámetros que quiero usar
params_grid = {'n_estimators': [50, 100], #nº de etapas de boosting
                        'learning_rate': [0.01, 0.1], #reduce la contribucion de cada arbol por este valor 
                        'max_features': [4, 5], #nº de variables a tener en cuenta para las divisiones
                        'min_samples_split': [5, 10]} #nº mínimo de observaciones necesarias para dividir un nodo interno (n.minobsinnode en R)

                
#Cantidad de splits para el Cross Validation
folds=8

#Regresor
gb_model_rd_search = ensemble.GradientBoostingRegressor()

#Metrica que quiero optimizar MSE
scorer_fn = make_scorer(sklearn.metrics.mean_squared_error)

#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=gb_model_rd_search,
                              param_distributions = params_grid,
                              scoring=scorer_fn,
                              n_iter=n, cv=folds, random_state=5) 


#Busco los hiperparamtros que optimizan MSE
randomcv.fit(x_train,y_train)

RandomizedSearchCV(cv=8, estimator=GradientBoostingRegressor(),
                   param_distributions={'learning_rate': [0.01, 0.1],
                                        'max_features': [4, 5],
                                        'min_samples_split': [5, 10],
                                        'n_estimators': [50, 100]},
                   random_state=5, scoring=make_scorer(mean_squared_error))

In [148]:
#Mejores hiperparametros
print(randomcv.best_params_)

#Mejor métrica
mse = randomcv.best_score_
print("MSE en datos de entrnamiento: " + str((mse)))
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))

{'n_estimators': 50, 'min_samples_split': 10, 'max_features': 4, 'learning_rate': 0.01}
MSE en datos de entrnamiento: 11457643426.571682
RMSE en datos de entrnamiento: 107040.38222358738


In [149]:
gb_model = ensemble.GradientBoostingRegressor().set_params(**randomcv.best_params_)
gb_model.fit(x_train, y_train)

GradientBoostingRegressor(learning_rate=0.01, max_features=4,
                          min_samples_split=10, n_estimators=50)

In [150]:
from sklearn.metrics import mean_squared_error

y_pred = gb_model.predict(x_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred, squared=True)

print("MSE en datos de test: " + str(mse))
print("RMSE en datos de test: " + str(rmse))

MSE en datos de test: 11491902723.986319
RMSE en datos de test: 107200.29255550714


In [151]:
#Atributos considerados y su importancia
sorted(list(zip(x_train.columns.to_list(), gb_model.feature_importances_)), key=lambda x: -x[1])

[('cp_2', 0.614391727078597),
 ('cp_1', 0.23665294493163597),
 ('cp_6', 0.08900036416810676),
 ('cp_4', 0.04265829701224729),
 ('cp_3', 0.009207258376270377),
 ('cp_5', 0.008089408433142629)]

Volvemos a buscar los hiperparametros que optimicen el MSE, y usamos el RMSE para analizar la performnce

Nuevamente vemos que la performance en el conjunto de entrenamiento y evaluación es similar. No hay overfitting, pero las métricas vuelven a dar un poco más elevadas con respecto a los otros dos algoritmos

La performance tanto con el dataset reducido, como en el dataset sin reducir fue muy similar. Se puede apreciar una leve mejora en la métricas en modelo que usa el dataset sin reducir. La mayor diferencia la notamos en el tiempo de ejecución, donde el modelo que usa el dataset sin reducir se ejcutó en menor tiempo

Guardamos el modelo

In [152]:
filename = "gradient_boost_reducido.sav"
pickle.dump(gb_model, open(filename, 'wb'))

<ipython-input-152-597caa2055c8>:2: ResourceWarning: unclosed file <_io.BufferedWriter name='gradient_boost_reducido.sav'>
  pickle.dump(gb_model, open(filename, 'wb'))


## Conclusiones

El algoritmo que peor performa es claramente el de GradientBoost. Entre los tres algoritmos, es el que peores métricas produce. Además es el que más tiempo tarda en ejecutar. 

XGBoost y KNN tienen performances similares, con KNN mostrando una pequeña mejoría en las métricas. Sin embargo, el tiempo de ejecución de KNN es menor. Por lo tanto, consideramos que los modelos de KNN son los mejores.

Como mencionamos más arriba, el modelo de KNN que usa el dataset reducido es levemente más preciso que el que usa el dataset original, y es un poco más rápido a la hora de ejecutarse. De esta forma, nos parece que el mejor modelo es el de KNN que utiliza el dataset reducido. Sin embargo, el resto de los modelos de KNN y XGBoost le siguen de cerca.
